In [ ]:
#pip install selenium
#pip install webdriver-manager

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time

# Ouvrir le navigateur
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

url = "https://webstat.banque-france.fr/fr/catalogue/mir1/MIR1.Q.FR.R.A22FRF.W3.R.A.2254FR.EUR.N"
driver.get(url)
time.sleep(5)  # attendre le JS

# Chercher le tableau
tables = driver.find_elements(By.TAG_NAME, "table")
if len(tables) == 0:
    print("Aucun tableau trouvé sur la page.")
    driver.quit()
    exit()

table = tables[0]  # on prend le premier tableau
rows = []

# Récupérer toutes les lignes <tr>
for tr in table.find_elements(By.TAG_NAME, "tr"):
    cells = tr.find_elements(By.TAG_NAME, "th") + tr.find_elements(By.TAG_NAME, "td")
    row = [cell.text.strip() for cell in cells]
    if row:  # ignorer les lignes vides
        rows.append(row)

driver.quit()

# Détecter automatiquement l'en-tête
header = rows[0]  # première ligne comme colonnes
data_rows = rows[1:]

# Créer DataFrame
df = pd.DataFrame(data_rows, columns=header)

# Détecter les colonnes "Période" et "Valeur" si elles existent
for col in df.columns:
    if "Période" in col:
        df["Période"] = df[col]
    if "Valeur" in col:
        df["Valeur"] = pd.to_numeric(df[col].str.replace(",", "."), errors="coerce")


Scraping terminé ! 35 lignes récupérées et CSV sauvegardé.


In [16]:
df.to_csv('credit_rate.csv')
